# Summary
Resaves the StatCan dissemination areas and Ookla tile overlay data as a faster to load *.pickle file.
In addition to loading/resaving the file, it adds some statistical information on whether the 
area is within a population center and population information.

In [1]:
import src.config

from src.datasets.loading import statcan, ookla

import numpy as np 
import pandas as pd
import geopandas as gp

import gc

In [2]:
import pickle 
import src.config

output_dir = src.config.DATA_DIRECTORY / 'processed' 
output_dir.mkdir(exist_ok = True)
feature_pickle = output_dir / 'feature_table.pickle'
all_tiles_pickle = output_dir / 'full_tileset.pickle'

In [3]:
da_pops = statcan.dissemination_areas_populations()
da_pops['DAUID'] = da_pops.DAUID.astype(str)

/home/jovyan/src/datasets/loading/statcan.py:234: DtypeWarning: Columns (3,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(POP_FILE)


In [4]:
speed_data = ookla.speed_data()
speed_data['quadkey'] = speed_data.quadkey.astype(str)

In [5]:
overlay_das_tiles = gp.read_file(src.config.OVERLAYS_DIR / 'tile_das_overlay')
overlay_das_tiles['DAUID'] = overlay_das_tiles.DAUID.astype(str)
overlay_das_tiles['quadkey'] = overlay_das_tiles.quadkey.astype(str)

In [6]:
gc.collect()
popctrs = statcan.boundary('population_centres')
p = popctrs.loc[:,['PCUID', 'PCNAME', 'PCTYPE', 'PCPUID', 'PCCLASS','geometry']]
overlay_das_tiles = overlay_das_tiles.sjoin(p, how='left')

overlay_das_tiles = overlay_das_tiles.merge(da_pops, on="DAUID", how='left')
gc.collect()

0

In [7]:
overlay_das_tiles = overlay_das_tiles.loc[lambda s:s.quadkey !='nan']

NameError: name 'overaly_das_tiles' is not defined

In [ ]:
overlay_das_tiles.info()

In [ ]:
with open(all_tiles_pickle, 'wb') as f:
    pickle.dump(overlay_das_tiles, f)